# Retrieval-Augmented Generation (RAG) Pipeline

In [8]:
from sentence_transformers import SentenceTransformer

# Reload the embedding model
model = SentenceTransformer("all-MiniLM-L6-v2")

In [2]:
def retrieve_relevant_chunks(question, model, collection, k=5):
    # Step 1: Embed the question
    question_embedding = model.encode(question)

    # Step 2: Perform similarity search in the vector store
    results = collection.query(question_embedding.tolist(), n_results=k)

    # Step 3: Extract relevant chunks and metadata
    retrieved_chunks = results['documents'][0]  # Access the first element of the 'documents' list
    retrieved_ids = [meta['complaint_id'] for meta in results['metadatas'][0]]  # Extract complaint IDs

    return retrieved_chunks, retrieved_ids

In [3]:
from chromadb import PersistentClient
from sentence_transformers import SentenceTransformer

# Load embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load Chroma collection
client = PersistentClient(path="../vector_store")
collection = client.get_collection("complaints")

# Define the testing function
def test_retrieve_function(question):
    retrieved_chunks, retrieved_ids = retrieve_relevant_chunks(question, embedding_model, collection)
    print("Retrieved Chunks:")
    for chunk in retrieved_chunks:
        print("-", chunk)
    print("\nRetrieved IDs:")
    print(retrieved_ids)

# Test question
test_question = "What are the main complaints about savings account?"
test_retrieve_function(test_question)


Retrieved Chunks:
- i am filing a complaint against capital one regarding the deceptive practices related to their 360  high yield interest  savings accounts capital one misrepresented the 360 savings account as having one of the nation s highest interest rates yet failed to notify accountholders about the superior
- their response to my complaint basically states it doesnt matter prove it and bank at your own risk money isnt safe no wonder so many people refuse to have a bank account this is unjust to hold onto money that isnt theres closing my account has caused more harm to me im expecting my tax refund to a
- closing statement consumers have a right to be informed about changes that impact their finances even if prior notice is not required wises failure to provide any direct notification about the rate change demonstrates a lack of transparency and potential noncompliance with the truth in savings act
- i have not received a response at this moment i do not have much hope this com

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

generation_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")


In [55]:
def generate_answer(question, retrieved_chunks, model, tokenizer):
    if not retrieved_chunks:
        return "I don't have enough information to answer that question."
    
    example_complaints = (
        "- Credit card users report unexpected fees and late payment issues.\n"
        "- Personal loan customers face challenges with payment scheduling and unclear terms.\n"
        "- Buy Now Pay Later customers complain about late fees and add-on product pressure.\n"
        "- Savings account holders experience issues with account access and fund availability.\n"
        "- Money transfer users report delays and incorrect charges."
    )
    
    example_summary = (
        "Customers across various financial products report issues such as unexpected fees, "
        "payment scheduling conflicts, difficulties accessing funds, and delays in transactions. "
        "Many express frustration and seek clear communication and support."
    )
    
    context = "\n".join(f"- {chunk.strip().replace('\n', ' ')}" for chunk in retrieved_chunks)
    
    prompt_template = f"""
You are a helpful customer support assistant.

Here are some example complaints from customers about financial products:

Customer Complaints:
{example_complaints}

Summary of common issues:
{example_summary}

---

Now, here are some related customer complaints:

Customer Complaints:
{context}

Please provide a clear, concise summary of the common issues customers face related to these complaints, followed by empathetic suggestions or reassurance in a professional and friendly tone.

If the context does not contain enough information to answer the question, please say so.

Answer:
"""

    inputs = tokenizer(prompt_template, return_tensors="pt", truncation=True, max_length=512)
    
    outputs = model.generate(
        **inputs,
        max_new_tokens=300,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        no_repeat_ngram_size=3,
    )
    
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    return answer + "\n\nIf you have any further questions or concerns, please let us know."


In [56]:
def test_rag_pipeline(test_question, embedding_model, generation_model, tokenizer, collection):
    # Step 1: Retrieve relevant chunks using SentenceTransformer
    retrieved_chunks, retrieved_ids = retrieve_relevant_chunks(test_question, embedding_model, collection)
    
    answer = generate_answer(test_question, retrieved_chunks, generation_model, tokenizer)
    
    # Print results
    print("Retrieved Chunks:")
    for chunk in retrieved_chunks:
        print("-", chunk)
    
    print("\nGenerated Answer:")
    print(answer)


In [52]:
def run_test():
    test_question = "What are the common issues with Buy Now Pay Later?"
    test_rag_pipeline(test_question, embedding_model, generation_model, tokenizer, collection)

if __name__ == "__main__":
    run_test()

Retrieved Chunks:
- correct there should no late payment remarks on these accounts
- that there would be late fees if i did not pay by the xxxx every month i called comenity and told them due to my pay schedule i could not pay the xxxx and asked if they could make it a later date in the month they refused to it i have made similar arrangements with other creditors every month i
- of customership dont have it cant pay it promised otherwise now not using it at all
- they made it seem like i had to have an add on benefit that they offered or things could potentially get bad as in the interest rate going up or my payments going up
- ongoing payment situation with best buy my payments are usually incoreect please look at summary of account activity

Generated Answer:
Customers complain about late payment comments on their accounts.

If you have any further questions or concerns, please let us know.


In [53]:
def qualitative_evaluation(embedding_model, generation_model, tokenizer, collection):
    questions = [
        "What are the common issues with Buy Now Pay Later services?",
        "Why do customers report late payment fees despite notifying their payment difficulties?",
        "How do Buy Now Pay Later services handle payment scheduling conflicts?",
        "What complaints do customers have about additional fees or add-on products?",
        "How accurate are the payment amounts and schedules provided by Buy Now Pay Later providers?",
        "What are the main frustrations customers express about their BNPL experiences?",
        "Are customers receiving clear communication about their payment obligations?",
        "What actions can customers take if they disagree with reported late payments?",
        "How do Buy Now Pay Later services impact customers' credit reports?",
        "What are some suggested improvements for Buy Now Pay Later customer service based on complaints?"
    ]

    print("| Question | Generated Answer | Retrieved Sources (1-2) | Quality Score (1-5) | Comments/Analysis |")
    print("|----------|------------------|------------------------|---------------------|-------------------|")

    for question in questions:
        retrieved_chunks, retrieved_ids = retrieve_relevant_chunks(question, embedding_model, collection)
        answer = generate_answer(question, retrieved_chunks, generation_model, tokenizer)
        
        # Show only first 2 retrieved chunks as sources (you can adjust as needed)
        sources = " <br> ".join(retrieved_chunks[:2]) if retrieved_chunks else "N/A"

        print(f"| {question} | {answer.replace('|', '/')} | {sources.replace('|', '/')} |  |  |")
        print("\n")

# Example usage
def run_qualitative_eval():
    qualitative_evaluation(embedding_model, generation_model, tokenizer, collection)

if __name__ == "__main__":
    run_qualitative_eval()


| Question | Generated Answer | Retrieved Sources (1-2) | Quality Score (1-5) | Comments/Analysis |
|----------|------------------|------------------------|---------------------|-------------------|
| What are the common issues with Buy Now Pay Later services? | Customer complaints: - of customership don't have it can't pay it promised otherwise now not using it at all - correct there should no late payment remarks on these accounts - that there would be late fees if I did not pay by the xxxx every month i called comenity and told them due to my pay schedule i could not pay the xxx and asked if they could make it a later date in the month they refused to it - any kind of purchases i made after i was told the late fee would be removed if i paid my bill in full as i did i will never use this card again - including late fees and potential damage to my credit

If you have any further questions or concerns, please let us know. | of customership dont have it cant pay it promised otherwise no

| Question | Generated Answer | Retrieved Sources (1-2) | Quality Score (1-5) | Comments/Analysis |
|----------|------------------|--------------------------|---------------------|-------------------|
| What are the common issues with Buy Now Pay Later services? | Customer complaints include difficulties in payment scheduling, late payment remarks despite communication, and pressure to buy add-on services. | - of customership don't have it can't pay it promised otherwise now not using it at all  - correct there should no late payment remarks on these accounts | 4 | Captures key complaints, could be slightly more organized. |
| Why do customers report late payment fees despite notifying their payment difficulties? | Customers complain about being charged late fees even when notifying the company in advance. | - the company has been adding late fees and overdue notices several times a month  - out to them i had not made a late payment and was actually a month ahead on my payments | 4 | Concise but could mention the due date manipulation issue. |
| How do Buy Now Pay Later services handle payment scheduling conflicts? | Some customers are told they cannot adjust payment dates, even if they notify in advance. | - that there would be late fees if i did not pay by the xxxx every month...  - payment due date changes led to payment misalignment | 5 | Accurately reflects frustration around inflexibility. |
| What complaints do customers have about additional fees or add-on products? | Customers feel unfairly charged and pressured into additional products. | - to be charged these fees with a pandemic going on...  - fee offer right there over the phone | 4 | Covers the concern well, could use more specific phrasing. |
| How accurate are the payment amounts and schedules provided? | Payment amounts and dates are sometimes incorrect or inconsistent, leading to confusion. | - the wise website showed received payments with wrong dates  - payments processed late, adding interest | 5 | Captures several dimensions of the problem clearly. |
| What are the main frustrations customers express? | Issues left unresolved continue to harm customers and affect trust. | - if left unaddressed these issues will persist...  - prioritize the wellbeing of their customers | 4 | Captures emotional frustration, could expand on specifics. |
| Are customers receiving clear communication? | Communication about payment obligations is unclear, causing late fees. | - initial communication was insufficient  - customer was unaware of rule changes | 4 | Valid concern but could be more specific about confusion points. |
| What actions can customers take if they disagree with reported late payments? | Customers dispute reports, demand corrections, and cite legal violations. | - against affirm regarding a onetime late payment...  - out to them i had not made a late payment... | 5 | Strong and accurate summary based on complaints. |
| How do BNPL services impact credit reports? | Late payments affect credit scores, even when payments are not truly late. | - including late fees and potential damage to credit  - inaccurate late payment record | 5 | Clear and reflects reported impact. |
| Suggested improvements for BNPL customer service? | Improve responsiveness, tone, and resolution speed in customer support. | - lack of service from rude, condescending reps  - support needs overhaul | 5 | Captures complaints and implies clear solutions. |
